In [3]:
from pyspark.sql import SparkSession

# Create a spark session 
spark = (
    SparkSession.builder.appName("Yellow Preprocessing")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/19 19:53:27 WARN Utils: Your hostname, MIchaels-HP resolves to a loopback address: 127.0.1.1; using 172.20.216.58 instead (on interface eth0)
23/08/19 19:53:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/19 19:53:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/19 19:53:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/08/19 19:53:30 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
from pyspark.sql import functions as F
import scipy.stats as sps

Gather initial information about our 2022 TLC data

In [5]:
yellow_sdf = spark.read.parquet('../data/raw/yellow_tlc_data/')

In [9]:
yellow_count_pre = yellow_sdf.count()
print('Number of trips in the original data is:', yellow_count_pre)

Number of trips in the original data is: 19838515


In [10]:
# now, we want to ensure everything has consistent casing 
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in yellow_sdf.columns]
yellow_sdf = yellow_sdf.select(*consistent_col_casing)

In [11]:
#Remove features from our dataframe that will have no use later in modelling or analysis
yellow_sdf = yellow_sdf.drop("store_and_fwd_flag", "vendorid")

In [18]:
#Sort the sdf by datetime so we can see if there are any trips taken before 2022-07-01 00:00:00 or after 2022-12-31 23:59:59
yellow_sdf = yellow_sdf.sort("tpep_pickup_datetime")
yellow_sdf.show(5)

yellow_sdf = yellow_sdf.sort(F.desc("tpep_pickup_datetime"))
yellow_sdf.show(5)

+--------------------+---------------------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|ratecodeid|pulocationid|dolocationid|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------------------+---------------------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
| 2001-01-01 00:03:14|  2001-01-01 01:12:47|            1.0|        20.77|       2.0|         132|         230|           2|       52.0|  0.0|    0.5|       0.0|        6.55|                  0.3|       61.85|                 2.5|        0.0|
| 2001-01-01 00:27:45|  2001

+--------------------+---------------------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|ratecodeid|pulocationid|dolocationid|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------------------+---------------------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
| 2023-01-01 00:31:23|  2023-01-01 00:43:25|            1.0|         3.06|       1.0|         239|          41|           1|       15.6|  1.0|    0.5|      4.12|         0.0|                  1.0|       24.72|                 2.5|        0.0|
| 2023-01-01 00:05:11|  2023

From this output we can see we do need to filter the dataframe by the afformentioned dates

In [19]:
#Filter by date, the amount of passengers, the fare amount and trip distance. 
yellow_sdf = yellow_sdf.filter(F.col("tpep_pickup_datetime")>"2022-07-01 00:00:00")\
                       .filter(F.col("tpep_pickup_datetime")<"2022-12-31 23:59:59")\
                       .filter(F.col("passenger_count")>0)\
                       .filter(F.col("fare_amount")>2.5)\
                       .filter(F.col("trip_distance")>=0.25)


In [20]:
def get_sdf_stats (sdf, id_col):
    """Function coagulates the mean and standard deviation of 
    the input columns id_col and returns these in a dictionary"""

    stats = {}

    for c in id_col:
        stats[c] = sdf.select(F.mean(c), F.stddev(c))
    return stats

In [21]:
#In order to identify outliers amongst the data, find the mean and standard deviation of trip distance and total amount
stats_columns = ['trip_distance', 'total_amount']
stats = get_sdf_stats(yellow_sdf, stats_columns)
stats

{'trip_distance': +------------------+--------------------------+
 |avg(trip_distance)|stddev_samp(trip_distance)|
 +------------------+--------------------------+
 | 3.725231711582681|         56.77742080177889|
 +------------------+--------------------------+,
 'total_amount': +-----------------+-------------------------+
 |avg(total_amount)|stddev_samp(total_amount)|
 +-----------------+-------------------------+
 | 22.3164832469607|       17.760110677748635|
 +-----------------+-------------------------+}

In [22]:
#Use these statistics to identify the 99th percentile, assuming a normal distribution,  as our maximum and filter any instancers with values greater than these
trip_mean = stats['trip_distance'].head()[0]
trip_stddev = stats['trip_distance'].head()[1]

amount_mean = stats['total_amount'].head()[0]
amount_stddev = stats['total_amount'].head()[1]

trip_distance_limit = trip_mean + sps.norm.ppf(0.99)*trip_stddev
total_amount_limit = amount_mean +sps.norm.ppf(0.99)*amount_stddev

yellow_sdf = yellow_sdf.filter(F.col("total_amount")<total_amount_limit)\
                       .filter(F.col("trip_distance")<trip_distance_limit)

In [25]:
#now that we have use total amount to remove any outliers that have been tabulated from tips, fare amount, etc. We can drop it as we will not be using it in our future analysis.
#And since Spark.ML cannot handle NULL types we will remove them all as well

yellow_clean_sdf = yellow_sdf.drop("total_amount")\
                       .dropna('any')

In [27]:
#Now we find out ho many instances were droped from the orginal data

yellow_count_post = yellow_clean_sdf.count()
pre_post_diff = yellow_count_pre - yellow_count_post

print("Number of instances after filtering:", yellow_count_post)
print("Number of instances removed:", pre_post_diff)

Number of instances after filtering: 17352747
Number of instances removed: 2485768


Now save our data to be used for feature engeineering

In [28]:
yellow_clean_sdf.write.mode('overwrite').parquet('../data/landing/yellow_tlc_clean')

23/08/19 20:18:40 WARN MemoryManager: Total allocation exceeds 95.00% (914,961,192 bytes) of heap memory
Scaling row group sizes to 97.39% for 7 writers
23/08/19 20:18:40 WARN MemoryManager: Total allocation exceeds 95.00% (914,961,192 bytes) of heap memory
Scaling row group sizes to 85.21% for 8 writers
23/08/19 20:18:52 WARN MemoryManager: Total allocation exceeds 95.00% (914,961,192 bytes) of heap memory
Scaling row group sizes to 97.39% for 7 writers
